In [8]:
RESULTS_PATH_WIKI =  ''
PDF_DIR = ''
RESULTS_PATH_GIT = ''
root_wiki = ''
root_git = ''

# Absolute Error (AE) box plots

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
def plot_box_group(df: pd.DataFrame | str, label_dict: dict, y_lim=(-0.01,1), label_y: str='AE', out_pdf: str=None, FONTSIZE:int=14, fig_size=(8,6)) -> None:
    if isinstance(df, str):
        df = pd.read_csv(df)
    plt.figure(figsize=fig_size)
    plt.ylim(y_lim[0],y_lim[1])
    plt.rcParams.update({'font.size': FONTSIZE})
    old_col_names = list(label_dict.keys())
    new_df = pd.DataFrame({label_dict[k]:df[k] for k in old_col_names})
    ax = new_df.boxplot(showfliers=False, whis=[0, 100], fontsize=FONTSIZE,showmeans=True, meanline=True, medianprops=dict(color='black'), boxprops=dict(color='black'), whiskerprops=dict(color='black'))    
    
    means = new_df.mean()
    for i, mean in enumerate(means):
        ax.text(i + 1, mean, f'{mean:.3f}', ha='center', va='bottom', color='green')

    # plt.xlabel('Approach', fontsize=FONTSIZE)
    plt.ylabel(label_y, fontsize=FONTSIZE)
    
    plt.grid(False)

    plt.tick_params(left = True) 
    if isinstance(out_pdf, str):
        plt.tight_layout()
        plt.savefig(out_pdf, format="pdf")#, bbox_inches="tight")
    plt.show()

In [10]:
runtimes_wikitables = pd.read_csv(RESULTS_PATH_WIKI)
runtimes_gittables = pd.read_csv(RESULTS_PATH_GIT)
runtimes_gittables['jsim_AE'] = abs(runtimes_gittables['jsim'] - runtimes_gittables['a%'])

In [ ]:
import pickle
evaluation_path_raw = ''
with open('', 'rb') as f:
    re_eval_armadillo_wiki = pickle.load(f)
re_eval_armadillo_wiki

In [79]:
new_overlaps = re_eval_armadillo_wiki['predictions']
runtimes_wikitables['new_col'] = [float(n) for n in new_overlaps]

In [ ]:
runtimes_wikitables['new_col']

In [45]:
runtimes_wikitables['armadillo_wikilast_ae'] = abs(runtimes_wikitables['new_col']-runtimes_wikitables['a%'])

In [ ]:
new_overlaps.shape
i = 0
for k in new_overlaps:
    if i==5:
        break
    print(k)
    i+=1

In [ ]:
re_eval_armadillo_wiki

In [ ]:
runtimes_wikitables[['r_id','s_id','a%','armadillo_wikilast_overlap_pred']]

In [ ]:
from tqdm import tqdm
for r in tqdm(range(runtimes_wikitables.shape[0])):
    label = runtimes_wikitables.iloc[r].loc['a%']
    

In [ ]:
#Wiki
plot_box_group(runtimes_wikitables, FONTSIZE=17, y_lim=(-0.05, 1.05), fig_size=(8,4), label_dict={'armadillo_g_w_ae':'Armadillo\nG','armadillo_w_w_ae':'Armadillo\nW','jsim_AE':'Jaccard','roberta_tables_300_300_gittables_ae':'RoBERTa\nT-G','roberta_tables_300_300_wikilast_ae':'RoBERTa\nT-W'}, label_y='AE',out_pdf=PDF_DIR+'box_plots_wiki.pdf')

In [ ]:
#Git
runtimes_gittables['jsim_AE'] = abs(runtimes_gittables['jsim']-runtimes_gittables['a%'])
plot_box_group(runtimes_gittables, FONTSIZE=17, y_lim=(-0.05, 1.05),fig_size=(8,4), label_dict={'armadillo_g_g_ae':'Armadillo\nG','armadillo_w_g_ae':'Armadillo\nW','jsim_AE':'Jaccard','roberta_t_g_g_ae':'RoBERTa\nT-G','roberta_t_w_g_ae':'RoBERTa\nT-W'}, label_y='AE',out_pdf=PDF_DIR+'box_plots_git.pdf')

In [ ]:
runtimes_wikitables.describe()

In [ ]:
runtimes_wikitables['jaccard_bag_smaller_times'].sum()

In [ ]:
runtimes_gittables['jaccard_bag_smaller_times'].sum()

In [ ]:
runtimes_gittables.describe()

# Mean Absolute Error (MAE) per overlap interval

In [123]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns

def compare_models_hist(data: pd.DataFrame | str, approaches: dict, bin_criterion: str='a%', bins_name: str='Correct Label', out_pdf: str=None, font_scale: float=1.45, n_col: int=4, fig_size=(8,6)) -> None:
    if isinstance(data, str):
        data = pd.read_csv(data)
    ranges = f'{bins_name} Range' 
    new_data = {
        ranges:[],
        'Approach':[],
        'MAE':[]
    }
    for i in range(1, 11, 1):
        i /= 10
        prev = round(i-0.1, 2)
        t = data[data[bin_criterion] >= prev]
        if i == 1:
            t = t[t[bin_criterion] <= i]
        else:
            t = t[t[bin_criterion] < i] 
        curr =  f'[{prev},\n{i}]'
        for k in approaches.keys():
            new_data['Approach'].append(approaches[k])
            new_data[ranges].append(curr)
            new_data['MAE'].append(round(np.mean(t[k]),2))
    plt.figure(figsize=fig_size)
    df = pd.DataFrame(new_data)
    sns.set_theme(font_scale=font_scale, style="whitegrid")
    ax = sns.barplot(data=df, x=ranges, y='MAE', hue='Approach')
    plt.grid(False)
    plt.tick_params(left = True) 
    plt.legend(bbox_to_anchor=(0.5, 1.29), loc='upper center', ncol=n_col)
    if isinstance(out_pdf, str):
        plt.tight_layout()
        plt.savefig(out_pdf, format="pdf")

In [124]:
runtimes_wikitables = pd.read_csv(RESULTS_PATH_WIKI)
runtimes_gittables = pd.read_csv(RESULTS_PATH_GIT)

In [ ]:
#Wiki
compare_models_hist(n_col=3, font_scale=1.01,fig_size=(6,4), data=runtimes_wikitables,approaches={'armadillo_g_w_ae':'Armadillo-G','armadillo_w_w_ae':'Armadillo-W','jsim_AE':'Jaccard','roberta_tables_300_300_gittables_ae':'RoBERTa-T-G','roberta_tables_300_300_wikilast_ae':'RoBERTa-T-W'},out_pdf=PDF_DIR+'mae_per_bin_wiki.pdf')

In [ ]:
runtimes_gittables.iloc[:,:15]


In [ ]:
runtimes_gittables['jsim_AE'] = abs(runtimes_gittables['jsim']-runtimes_gittables['a%'])
compare_models_hist(n_col=3, font_scale=1.01,fig_size=(6,4),data=runtimes_gittables,approaches={'armadillo_g_g_ae':'Armadillo-G','armadillo_w_g_ae':'Armadillo-W','jsim_AE':'Jaccard','roberta_t_g_g_ae':'RoBERTa-T-G','roberta_t_w_g_ae':'RoBERTa-T-W'},out_pdf=PDF_DIR+'mae_per_bin_git.pdf')

# Cumulative execution time per model

In [25]:
# from Code._performance_overlap_computation import *
from tqdm import tqdm
import pandas as pd
def reduce_range(l_in: list) -> list:
    return [k/1000 for k in l_in]

def show_scatter_t_exec_sloth_arm(results: str | pd.DataFrame, x_label: str='tot_area', logx: bool=True, logy: bool=True, output_pdf: str=None, x_axis: str=None, font_size=15, zoom:int =None) -> None:
    if isinstance(results, str):
        data = pd.read_csv(results)
    else:
        data = results
    
    plt.figure()
    ax_scatter = plt.subplot()
    # plt.ticklabel_format(axis='both', style='sci', scilimits=(3,3), useMathText=True)
    if x_axis == None:
        x = t_execs_sloth
    else:
        x = reduce_range(data[x_axis])
    if zoom!=None:
         plt.ylim(0, zoom)
    try:
        t_execs_sloth = reduce_range(data['sloth_time'])
        ax_scatter.scatter(x, t_execs_sloth, s=3, c='black', alpha=0.7, edgecolors='black', label='Sloth')
    except:
        pass
    try:
        t_execs_embdi_wikilast_overlap_time = reduce_range(data['embdi_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_embdi_wikilast_overlap_time, s=3, c='grey', alpha=0.7, edgecolors='grey', label='EmbDI')
    except:
        pass
    try:
        t_execs_jsim = reduce_range(data['jsim_time'])
        ax_scatter.scatter(x, t_execs_jsim, s=3, c='green', alpha=0.7, edgecolors='green', label='Jaccard')
    except:
        pass

    try:
        t_execs_armadillo_wikilast = reduce_range(data['armadillo_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_armadillo_wikilast, s=3, c='blue', alpha=0.7, edgecolors='blue', label='Armadillo')
    except:
        pass

    try:
        t_execs_turl_wikilast_overlap_time = reduce_range(data['turl_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_turl_wikilast_overlap_time, s=3, c='orange', alpha=0.7, edgecolors='orange', label='TURL')
    except:
        pass
    try:
        t_execs_roberta_tables_300_300_wikilast_overlap_time = reduce_range(data['roberta_tables_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_tables_300_300_wikilast_overlap_time, s=3, c='pink', alpha=0.7, edgecolors='pink', label='RoBERTa-T')
    except:
        pass
    try:
        t_execs_roberta_rows_300_300_wikilast_overlap_time = reduce_range(data['roberta_rows_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_rows_300_300_wikilast_overlap_time, s=3, c='brown', alpha=0.7, edgecolors='brown', label='RoBERTa-R')
    except:
        pass
    if zoom == None:
        ax_scatter.set_ylabel(f'Cumulative runtime (s)($10^3$)',fontsize=font_size)
        ax_scatter.set_xlabel(f'Number of samples ($10^3$)', fontsize=font_size)
    if logx:
        ax_scatter.set_xscale('log')
    if logy:
        ax_scatter.set_yscale('log')
    
    ax_scatter.tick_params(axis='both', which='major', labelsize=font_size)  


    ax_scatter.tick_params(axis='both', which='both', labelsize=font_size)
    if zoom == None:
        ax_scatter.legend(fontsize=font_size, scatterpoints=100,loc='upper center', bbox_to_anchor=(0.5, 1.35),
            ncol=3, fancybox=True, shadow=True)
    plt.grid(False)
    # plt.tight_layout()
    if isinstance(output_pdf, str):
        plt.savefig(output_pdf, format="pdf", bbox_inches="tight")
    plt.show()

def build_incremental_t_execs(results: pd.DataFrame | str, interval: int=100, sort: bool=True) -> None:
    if isinstance(results, str):
        results = pd.read_csv(results)
    if sort:
        try:
            results = results.sort_values(by='sloth_time', ascending=True)
        except:
            results = results.sort_values(by='total_time', ascending=True)
    df_out = {
        'n_samples':[],
        'sloth_time':[],
        'jsim_time':[],
        'cosine_similarity_overlap_time':[],

        'armadillo_wikilast_overlap_time':[],
        'embdi_wikilast_overlap_time':[],
        'turl_wikilast_overlap_time':[],
        'roberta_tables_300_300_wikilast_overlap_time':[],
        'roberta_rows_300_300_wikilast_overlap_time':[]
    }
    cur_sum = {
        'sloth_time':0,
        'jsim_time':0,
        'cosine_similarity_overlap_time':0,

        'armadillo_wikilast_overlap_time':0,
        'embdi_wikilast_overlap_time':0,
        'turl_wikilast_overlap_time':0,
        'roberta_tables_300_300_wikilast_overlap_time':0,
        'roberta_rows_300_300_wikilast_overlap_time':0
    }
    for r in tqdm(range(results.shape[0])):
        for k in cur_sum.keys():
            try:
                cur_sum[k] += results.iloc[r].loc[k]
            except:
                if k == 'sloth_time':
                    cur_sum[k] += results.iloc[r].loc['total_time']
                else:
                    print(f'K was {k}')
                    raise Exception
        if (r % interval) == 0:
            df_out['n_samples'].append(r+1)
            for k in cur_sum.keys():
                try:
                    df_out[k].append(cur_sum[k])
                except:
                    if k == 'sloth_time':
                        df_out[k].append(cur_sum['total_time'])
                    else: 
                        raise Exception
    df_out = pd.DataFrame(df_out)
    return df_out


In [26]:
def show_scatter_t_exec_sloth_arm_small(results: str | pd.DataFrame, x_label: str='tot_area', logx: bool=True, logy: bool=True, output_pdf: str=None, x_axis: str=None, font_size=15, zoom:int =10) -> None:
    if isinstance(results, str):
        data = pd.read_csv(results)
    else:
        data = results
    
    plt.figure()
    ax_scatter = plt.subplot()
    # plt.ticklabel_format(axis='both', style='sci', scilimits=(3,3), useMathText=True)
    if x_axis == None:
        x = t_execs_sloth
    else:
        x = reduce_range(data[x_axis])
    if zoom!=None:
         plt.ylim(-0.5, zoom)
    try:
        t_execs_sloth = reduce_range(data['sloth_time'])
        ax_scatter.scatter(x, t_execs_sloth, s=7, c='black', alpha=0.7, edgecolors='black', label='Sloth')
    except:
        pass
    try:
        t_execs_embdi_wikilast_overlap_time = reduce_range(data['embdi_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_embdi_wikilast_overlap_time, s=7, c='grey', alpha=0.7, edgecolors='grey', label='EmbDI')
    except:
        pass
    # try:
    #     t_execs_cosine = reduce_range(data['cosine_similarity_overlap_time'])
    #     ax_scatter.scatter(x, t_execs_cosine, s=3, c='red', alpha=0.7, edgecolors='red', label='Cosine')
    # except:
    #     pass
    try:
        t_execs_jsim = reduce_range(data['jsim_time'])
        ax_scatter.scatter(x, t_execs_jsim, s=3, c='green', alpha=0.7, edgecolors='green', label='Jaccard')
    except:
        pass

    try:
        t_execs_armadillo_wikilast = reduce_range(data['armadillo_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_armadillo_wikilast, s=7, c='blue', alpha=0.7, edgecolors='blue', label='Armadillo')
    except:
        pass

    try:
        t_execs_turl_wikilast_overlap_time = reduce_range(data['turl_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_turl_wikilast_overlap_time, s=7, c='orange', alpha=0.7, edgecolors='orange', label='TURL')
    except:
        pass
    try:
        t_execs_roberta_tables_300_300_wikilast_overlap_time = reduce_range(data['roberta_tables_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_tables_300_300_wikilast_overlap_time, s=7, c='pink', alpha=0.7, edgecolors='pink', label='RoBERTa-T')
    except:
        pass
    try:
        t_execs_roberta_rows_300_300_wikilast_overlap_time = reduce_range(data['roberta_rows_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_rows_300_300_wikilast_overlap_time, s=7, c='brown', alpha=0.7, edgecolors='brown', label='RoBERTa-R')
    except:
        pass
    if zoom == None:
        ax_scatter.set_ylabel(f'Cumulative runtime (s)($10^3$)',fontsize=font_size)
        ax_scatter.set_xlabel(f'Number of samples ($10^3$)', fontsize=font_size)
    
    # ax_scatter.tick_params(axis='both', which='major', labelsize=font_size)  


    # ax_scatter.tick_params(axis='both', which='both', labelsize=font_size)
    ax_scatter.set_yticklabels([])
    ax_scatter.set_xticklabels([])

    plt.grid(False)
    # plt.tight_layout()
    if isinstance(output_pdf, str):
        plt.savefig(output_pdf, format="pdf", bbox_inches="tight")
    plt.show()

In [27]:
def show_scatter_t_exec_sloth_arm_v2(results: str | pd.DataFrame, x_label: str='tot_area', logx: bool=True, logy: bool=True, output_pdf: str=None, x_axis: str=None, font_size=18, zoom:int =None,
                                     figsize: tuple=(11.0,7.0)) -> None:

    if isinstance(results, str):
        data = pd.read_csv(results)
    else:
        data = results
    
    plt.rcParams["figure.figsize"] = [figsize[0], figsize[1]]
    plt.rcParams["figure.autolayout"] = True

    plt.figure()
    ax_scatter = plt.subplot()
    # plt.ticklabel_format(axis='both', style='sci', scilimits=(3,3), useMathText=True)
    if x_axis == None:
        x = t_execs_sloth
    else:
        x = reduce_range(data[x_axis])
    if zoom!=None:
         plt.ylim(-4, zoom)
    try:
        t_execs_sloth = reduce_range(data['sloth_time'])
        ax_scatter.scatter(x, t_execs_sloth, s=3, c='black', alpha=0.7, edgecolors='black', label='Sloth')
    except:
        pass
    try:
        t_execs_embdi_wikilast_overlap_time = reduce_range(data['embdi_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_embdi_wikilast_overlap_time, s=3, c='grey', alpha=0.7, edgecolors='grey', label='EmbDI')
    except:
        pass
    # try:
    #     t_execs_cosine = reduce_range(data['cosine_similarity_overlap_time'])
    #     ax_scatter.scatter(x, t_execs_cosine, s=3, c='red', alpha=0.7, edgecolors='red', label='Cosine')
    # except:
    #     pass
    try:
        t_execs_jsim = reduce_range(data['jsim_time'])
        ax_scatter.scatter(x, t_execs_jsim, s=3, c='green', alpha=0.7, edgecolors='green', label='Jaccard')
    except:
        pass

    try:
        t_execs_armadillo_wikilast = reduce_range(data['armadillo_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_armadillo_wikilast, s=3, c='blue', alpha=0.7, edgecolors='blue', label='Armadillo')
    except:
        pass

    try:
        t_execs_turl_wikilast_overlap_time = reduce_range(data['turl_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_turl_wikilast_overlap_time, s=3, c='orange', alpha=0.7, edgecolors='orange', label='TURL')
    except:
        pass
    try:
        t_execs_roberta_tables_300_300_wikilast_overlap_time = reduce_range(data['roberta_tables_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_tables_300_300_wikilast_overlap_time, s=3, c='pink', alpha=0.7, edgecolors='pink', label='RoBERTa-T')
    except:
        pass
    try:
        t_execs_roberta_rows_300_300_wikilast_overlap_time = reduce_range(data['roberta_rows_300_300_wikilast_overlap_time'])
        ax_scatter.scatter(x, t_execs_roberta_rows_300_300_wikilast_overlap_time, s=3, c='brown', alpha=0.7, edgecolors='brown', label='RoBERTa-R')
    except:
        pass
    ax_scatter.set_ylabel(f'Cumulative runtime (s)($10^3$)',fontsize=font_size)
    ax_scatter.set_xlabel(f'Number of samples ($10^3$)', fontsize=font_size)

    ax_scatter.tick_params(axis='both', which='major', labelsize=font_size)  

    ax_scatter.tick_params(axis='both', which='both', labelsize=font_size)
    if zoom == None or zoom >= 70:
        ax_scatter.legend(fontsize=font_size, scatterpoints=100,loc='upper center', bbox_to_anchor=(0.5, 1.16),
            ncol=4, fancybox=True, shadow=True)
    plt.grid(False)
    # plt.tight_layout()
    if isinstance(output_pdf, str):
        plt.savefig(output_pdf, format="pdf", bbox_inches="tight")
    plt.show()

In [34]:
runtimes_wikitables = pd.read_csv(RESULTS_PATH_WIKI)
runtimes_gittables = pd.read_csv(RESULTS_PATH_GIT)

In [ ]:
runtimes_gittables

In [ ]:
runtimes_gittables['cosine_similarity_overlap_time'].sum()

In [ ]:
runtimes_gittables['armadillo_w_g_ae'].mean()

In [ ]:
runtimes_wikitables

In [ ]:
runtimes_wikitables['armadillo_g_w_ae'].mean()

In [ ]:
runtimes_wikitables.describe()[['sloth_time','jsim_time','armadillo_wikilast_overlap_time','embdi_wikilast_overlap_time','turl_wikilast_overlap_time',
                                'bert_tables_300_300_wikilast_overlap_time','bert_tables_anon_300_300_wikilast_overlap_time','roberta_tables_300_300_wikilast_overlap_time',
                                'roberta_tables_anon_300_300_wikilast_overlap_time','bert_rows_300_300_wikilast_overlap_time','roberta_rows_300_300_wikilast_overlap_time','jsim_AE',
                                'armadillo_wikilast_ae', 'embdi_wikilast_ae','turl_wikilast_ae','bert_tables_300_300_wikilast_ae','bert_tables_anon_300_300_wikilast_ae',
                                'roberta_tables_300_300_wikilast_ae','roberta_tables_anon_300_300_wikilast_ae','bert_rows_300_300_wikilast_ae','roberta_rows_300_300_wikilast_ae']]

In [ ]:
total_time_wiki = {}
total_time_wiki['Sloth_W'] = runtimes_wikitables['sloth_time'].sum()
total_time_wiki['Armadillo_W-W'] = runtimes_wikitables['armadillo_wikilast_overlap_time'].sum()
# total_time_wiki['Armadillo_G'] = runtimes_wikitables['armadillo_wikilast_overlap_time'].sum()
total_time_wiki['Cosine_similarity'] = runtimes_wikitables['cosine_similarity_overlap_time'].sum()

total_time_wiki['EmbDI_W'] = runtimes_wikitables['embdi_wikilast_overlap_time'].sum()
total_time_wiki['Jaccard'] = runtimes_wikitables['jsim_time'].sum()
# total_time_wiki['TURL-T-G'] = runtimes_wikitables[''].sum()
total_time_wiki['TURL-T-W'] = runtimes_wikitables['turl_wikilast_overlap_time'].sum()
# total_time_wiki['BERT-R-G'] = runtimes_wikitables[''].sum()
total_time_wiki['BERT-R-W'] = runtimes_wikitables['bert_rows_300_300_wikilast_overlap_time'].sum()
# total_time_wiki['BERT-T-G'] = runtimes_wikitables[''].sum()
total_time_wiki['BERT-T-W'] = runtimes_wikitables['bert_tables_300_300_wikilast_overlap_time'].sum()
total_time_wiki['BERT-T-W-N'] = runtimes_wikitables['bert_tables_anon_300_300_wikilast_overlap_time'].sum()
# total_time_wiki['RoBERTa-R-G'] = runtimes_wikitables[''].sum()
total_time_wiki['RoBERTa-R-W'] = runtimes_wikitables['roberta_rows_300_300_wikilast_overlap_time'].sum()
# total_time_wiki['RoBERTa-T-G'] = runtimes_wikitables[''].sum()
total_time_wiki['RoBERTa-T-W'] = runtimes_wikitables['roberta_tables_300_300_wikilast_overlap_time'].sum()
total_time_wiki['RoBERTa-T-W-N'] = runtimes_wikitables['roberta_tables_anon_300_300_wikilast_overlap_time'].sum()
total_time_wiki['Armadillo-G-W'] = runtimes_wikitables['armadillo_g_w_overlap_time'].sum()

total_time_wiki

In [ ]:
wikitables_inc_no_sort = build_incremental_t_execs(runtimes_wikitables, sort=False)

In [ ]:
show_scatter_t_exec_sloth_arm_small(wikitables_inc_no_sort, x_axis='n_samples', logx=False, logy=False, output_pdf=PDF_DIR+'cumulative_t_execs_no_sort_zoom.pdf', zoom=10)

In [ ]:
show_scatter_t_exec_sloth_arm_v2(wikitables_inc_no_sort, x_axis='n_samples', logx=False, logy=False, output_pdf=PDF_DIR+'cumulative_t_execs_no_sort.pdf',figsize=(10.3,8), zoom=79)

# NDCG score

In [ ]:
from Code._table_querying import *

In [11]:
def compute_ndcg_at_k(table_querying_arm_sloth: str | pd.DataFrame, query_set: str | set, outpath: str, k_max: int=100) -> pd.DataFrame:
    if isinstance(table_querying_arm_sloth, str):
        table_querying_arm_sloth = pd.read_csv(table_querying_arm_sloth)
    if isinstance(query_set, str):
        with open(query_set, 'rb') as f:
            query_set = pickle.load(f)
    
    out = {
        'k':[],
        'query_table':[],
        'Armadillo':[],
        'Jaccard':[],
        'Jaccard-B-U':[],
        'Jaccard-B-S':[],
        'BERT-R':[],
        'BERT-T':[],
        'RoBERTa-R':[],
        'RoBERTa-T':[],
        'TURL-T':[]
    }

    for k in [1,10,100,1000]:#range(0, k_max+1, 10):
        if k == 0:
            k=1
        print(f'Current k: {k}')
        for t in tqdm(query_set):
            out['k'].append(k)
            out['query_table'].append(t)
            curr_subset = table_querying_arm_sloth[table_querying_arm_sloth['r_id']==t]
            out['Armadillo'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['armadillo_g_g_overlap_pred']]).reshape(1,-1), k=k))
            out['Jaccard'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jsim']]).reshape(1,-1), k=k))
            out['Jaccard-B-U'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jaccard_bag_union_predictions']]).reshape(1,-1), k=k))
            out['Jaccard-B-S'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jaccard_bag_smaller_predictions']]).reshape(1,-1), k=k))
            out['BERT-R'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['bert_r_overlap_pred']]).reshape(1,-1), k=k))
            out['BERT-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['bert_t_overlap_pred']]).reshape(1,-1), k=k))
            out['RoBERTa-R'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['roberta_r_overlap_pred']]).reshape(1,-1), k=k))
            out['RoBERTa-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['roberta_t_overlap_pred']]).reshape(1,-1), k=k))
            out['TURL-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['turl_overlap_pred']]).reshape(1,-1), k=k))

    k = 10_000
    for t in query_set:
        out['k'].append(k)
        out['query_table'].append(t)
        curr_subset = table_querying_arm_sloth[table_querying_arm_sloth['r_id']==t]
        out['Armadillo'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['armadillo_g_g_overlap_pred']]).reshape(1,-1), k=k))
        out['Jaccard'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jsim']]).reshape(1,-1), k=k))
        out['Jaccard-B-U'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jaccard_bag_union_predictions']]).reshape(1,-1), k=k))
        out['Jaccard-B-S'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['jaccard_bag_smaller_predictions']]).reshape(1,-1), k=k))
        out['BERT-R'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['bert_r_overlap_pred']]).reshape(1,-1), k=k))
        out['BERT-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['bert_t_overlap_pred']]).reshape(1,-1), k=k))
        out['RoBERTa-R'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['roberta_r_overlap_pred']]).reshape(1,-1), k=k))
        out['RoBERTa-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['roberta_t_overlap_pred']]).reshape(1,-1), k=k))
        out['TURL-T'].append(ndcg_score(y_true=np.array(curr_subset[['a%']]).reshape(1,-1), y_score=np.array(curr_subset[['turl_overlap_pred']]).reshape(1,-1), k=k))
    out = pd.DataFrame(out)
    out.to_csv(outpath, index=False)
    return out

def compare_models_ndcg(data: pd.DataFrame | str, bin_criterion: str='k', bins_name: str='Correct Label', out_pdf: str=None, font_scale: float=0.7, n_cols_legend=4, position_legend=(0.5, 1.1), fig_size=(8,6)) -> pd.DataFrame:
    """Function to plot an histogram to compare performances of different models depending on their range of error 

    Args:
        data (pd.DataFrame | str): data frame containing the results
        bin_criterion (str, optional): parameter to generate the 10 bins, must be with values in [0,1]. Defaults to 'a%'.
        bins_name (str, optional): name of the bins. Defaults to 'AE'.
    """
    if isinstance(data, str):
        data = pd.read_csv(data)
    new_data = {
        'k':[],
        'Approach':[],
        'NDCG_Score':[]
    }
    for k in [1,10,100,1000,10_000]:
        t = data[data[bin_criterion] == k]
        if k == 10_000:
            k = '10k'
        new_data['Approach'].append('Armadillo')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['Armadillo']),2))
        
        new_data['Approach'].append('Jaccard')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['Jaccard']),2))

        # new_data['Approach'].append('Jaccard-B-U')
        # new_data['k'].append(k)
        # new_data['NDCG_Score'].append(round(np.mean(t['Jaccard-B-U']),2))

        new_data['Approach'].append('Jaccard-B-S')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['Jaccard-B-S']),2))

        # new_data['Approach'].append('BERT-R')
        # new_data['k'].append(k)
        # new_data['NDCG_Score'].append(round(np.mean(t['BERT-R']),2))

        # new_data['Approach'].append('BERT-T')
        # new_data['k'].append(k)
        # new_data['NDCG_Score'].append(round(np.mean(t['BERT-T']),2))

        new_data['Approach'].append('RoBERTa-R')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['RoBERTa-R']),2))

        new_data['Approach'].append('RoBERTa-T')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['RoBERTa-T']),2))

        new_data['Approach'].append('TURL-T')
        new_data['k'].append(k)
        new_data['NDCG_Score'].append(round(np.mean(t['RoBERTa-T']),2))


    plt.figure(figsize=fig_size)
    df = pd.DataFrame(new_data)
    sns.set_theme(font_scale=font_scale, style="whitegrid")
    ax = sns.barplot(data=df, x='k', y='NDCG_Score', hue='Approach')

    ax.tick_params(axis='both', which='major', labelsize=14)  # Set font size for major ticks
    #ax.tick_params(axis='both', which='major', labelsize=font_size)  # Set font size for major ticks

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc='upper center', bbox_to_anchor=position_legend, ncol=n_cols_legend)#len(handles))


    plt.grid(False)
    plt.tick_params(left = True) 
    if isinstance(out_pdf, str):
        plt.tight_layout()
        plt.savefig(out_pdf, format="pdf", bbox_inches="tight")
    return df

In [12]:
def prepare_triple_dataset(df):
    to_drop = []
    for r in tqdm(range(df.shape[0])):
        t = df.iloc[r]
        if pd.isna(t['a%']) and (t['seeds'] != 0):
            to_drop.append(r)
    df.drop(to_drop)
    df['a%'] = df['a%'].fillna(0)
    return df


In [14]:
path_querying_results = ''

querying_table_results = pd.read_csv(path_querying_results)

In [ ]:
# Not optimized times
times = {}
times['Sloth'] = querying_table_results['total_time'].sum()
times['Jaccard'] = querying_table_results['jsim_time'].sum()
times['BERT-T'] = querying_table_results['bert_t_overlap_time'].sum()
times['RoBERTa-R'] = querying_table_results['roberta_r_overlap_time'].sum()
times['RoBERTa-T'] = querying_table_results['roberta_t_overlap_time'].sum()
times['BERT-R'] = querying_table_results['bert_r_overlap_time'].sum()
times['Armadillo'] = querying_table_results['armadillo_g_g_overlap_time'].sum()
times['Jaccard-B-U'] = querying_table_results['jaccard_bag_union_times'].sum()
times['Jaccard-B-S'] = querying_table_results['jaccard_bag_smaller_times'].sum()
times['TURL'] = querying_table_results['turl_overlap_time'].sum()
times


In [16]:
table_querying_all = set(querying_table_results['r_id']).union(set(querying_table_results['s_id']))

In [ ]:
len(table_querying_all)

In [40]:
import datetime
from tqdm import tqdm
def get_emb_time(tables, times_path):
    with open(times_path, 'rb') as f:
        times = pickle.load(f)
    cumul = 0
    add_more=False
    for t in tqdm(tables):
        try:
            cumul+=times[t]['t_tot']
        except:
            cumul+=times[t]
            add_more=True
    if add_more:
        cumul+=6_000
    return str(datetime.timedelta(seconds = cumul))

In [ ]:
table_querying

In [ ]:
t_execs_root_dictionary = ''

emb_times = {}
emb_times['Armadillo'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_armadillo_querying.pkl')
emb_times['TURL'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_turl.pkl')
emb_times['BERT-R'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_bert_r.pkl')
emb_times['BERT-T'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_bert_t.pkl')

emb_times['RoBERTa-R'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_roberta_r.pkl')
emb_times['RoBERTa-T'] = get_emb_time(table_querying_all, t_execs_root_dictionary+'/t_execs_roberta_t.pkl')

emb_times


In [ ]:
querying_table_results = prepare_triple_dataset(querying_table_results)

In [ ]:
querying_table_results.keys()

In [ ]:
ndcg_directory = ''
compute_ndcg_at_k(querying_table_results, set(querying_table_results['r_id']), outpath=ndcg_directory+'/evaluation/charts/ndcg_score.csv')

In [ ]:
pdf_directory = ''
res = compare_models_ndcg(ndcg_directory+'/charts/ndcg_score.csv', fig_size=(6,4), position_legend=(0.5, 1.3), n_cols_legend=3, font_scale=1.1, out_pdf=pdf_directory+'/ndcg_score.pdf')

In [ ]:
res

# Overlap comp time area

In [174]:
from Code._performance_overlap_computation import *
def visualize_scatter_plot(exp_data_file: str | dict, logx: bool=True, logy: bool=True, out_pdf: str=None, font_size: int=10, fig_size=(8,8)) -> None:
    """visualize embedding generation time on the y axis and table area on the x axis

    Args:
        exp_data_file (str | dict): path to a file containing the data related to a "embed_all_no_paral" test or the dictionary containing the actual data
        logx (bool, opt): if True, the x axis is in logscale. Defaults to True.
        logy (bool, opt): if True, the y axis is in logscale. Defaults to False.
    """
    if isinstance(exp_data_file, str):
        with open(exp_data_file, 'rb') as f:
            data = pickle.load(f)
    else:
        data = exp_data_file

    keys = list(data.keys())

    areas = [data[k]['area'] for k in keys]
    t_execs = [data[k]['t_tot'] for k in keys]

    x = areas
    y = t_execs
    plt.figure()
    # Definisci la figura e gli assi per lo scatterplot
    fig, (ax_scatter, ax_kde) = plt.subplots(2, 1, figsize=fig_size, 
                                            gridspec_kw={'height_ratios': [3, 1]})
    ax_scatter.grid(False)
    ax_scatter.tick_params(left = True, bottom=True) 
    # Disegna lo scatterplot
    ax_scatter.scatter(x, y, s=3, c='orange', alpha=0.7, edgecolors='black')

    sns.histplot(
    data=x, ax=ax_kde,
    label='KDE',
    fill=True, common_norm=False,
    alpha=.5, linewidth=0, color='grey'
    )
    ax_scatter.set_ylabel('Embedding Time (s)', fontsize=font_size)
    
    if logx:
        ax_kde.set_xscale('log')    
        ax_scatter.set_xscale('log')
    if logy:
        ax_scatter.set_yscale('log')
    ax_kde.set_yscale('log')
    
    ax_kde.set_xlabel('Table Area', fontsize=font_size)
    ax_kde.set_ylabel('Number Of Samples', fontsize=font_size)

    ax_scatter.tick_params(axis='both', which='major', labelsize=font_size)  # Set font size for major ticks
    ax_kde.tick_params(axis='both', which='major', labelsize=font_size)  # Set font size for major ticks

    ax_scatter.tick_params(axis='both', which='both', labelsize=font_size)  # Set font size for tick labels
    ax_kde.tick_params(axis='both', which='both', labelsize=font_size)  # Set font size for tick labels

    # plt.tight_layout()
    plt.grid(False)
    plt.tick_params(left = True, bottom=True) 
    if isinstance(out_pdf, str):
        plt.savefig(out_pdf, format="pdf", bbox_inches="tight")

    plt.show()

In [ ]:
visualize_scatter_plot(root_git+'/WikiTables/dictionaries/embedding_dictionaries/t_execs_armadillo_w_w.pkl', logx=True, logy=True, out_pdf=None, font_size=15)

In [ ]:
visualize_scatter_plot(root_wiki+'/GitTables/dictionaries/embedding_dictionaries/t_execs_armadillo_g_g.pkl', logx=True, logy=True, out_pdf=None, font_size=15.5, fig_size=(8,5.5))